# Make Predictions on Test Data
1. Choose top 3 models from Notebook 4
2. Make predictions
3. Report results

In [2]:
import numpy as np
import pandas as pd
import psycopg2 as pg
import datetime as dt
import pickle
import ast
import os
import time
import itertools

from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from lib.Modeling import test_binary_level, test_multi_level

%matplotlib inline
import matplotlib.pyplot as plt

### set inputs

In [3]:
# model configurations

file_args = yaml.load(open('./conf/pipeline_args.yml','r'))

In [4]:
# create db connection objects

conn_str_file = './conf/db_conn_str.txt'

pg_conn_str = open(conn_str_file, 'r').read()
conn = pg.connect(pg_conn_str)

sqlalchemy_conn_str = open('./conf/sqlalchemy_conn_str.txt', 'r').read()
engine = create_engine(sqlalchemy_conn_str)

## Load data

In [ ]:
train_data = pickle.load(open('./data/train_test/final_training_data.pkl','rb'), protocol=4)

In [ ]:
test_data = pickle.load(open('./data/train_test/final_test_data.pkl','rb'), protocol=4)

In [ ]:
parameters = pickle.load(open('./data/train_test/final_parameters.pkl','rb'), protocol=4)

# Prediction

In [ ]:
metrics_df = pd.DataFrame(columns=['model','parameters','step','cluster','accuracy',
                                   'precision','recall','f1score','roc_auc','time_taken',
                                   'pos_neg_ratio'])

In [ ]:
for cluster in train_data['cluster'].unique():
    
    ## Predict binary target
    predictions, metrics = test_binary_level(train_data, test_data, cluster, parameters_binary['cluster'])
    metrics_df = metrics_df.append(pd.DataFrame(metrics, index=[0]))
    
    ## Use predictions from binary target for multi target
    multi_test = test_data.loc[(test_data['cluster'] == cluster)]
    multi_test['level_binary_predictions'] = predictions
    
    ## Predict multi level target
    predictions, metrics = test_binary_level(train_data, test_data, cluster, parameters_multi['cluster'])
    metrics_df = metrics_df.append(pd.DataFrame(metrics, index=[0]))

# Evaluate

In [ ]:
metrics_df = metrics_df[['model','parameters','step','cluster','accuracy','precision',
                         'recall','f1score','roc_auc','time_taken','train_size','pos_neg_ratio']]

In [ ]:
metrics_df